In [53]:
import pulp
import logging
import pandas as pd 

logging.getLogger().setLevel(level=logging.INFO)
logging.getLogger("yahoo_oauth").setLevel(level=logging.INFO)


from csh_fantasy_bot.bot import ManagerBot
from nhl_scraper.nhl import Scraper

from pulp import *

league_id = '403.l.41177'
week_number = 2
manager = ManagerBot(week_number, league_id=league_id)

my_scores = manager.my_team.scores()
manager.score_comparer.print_week_results(my_scores)


               G     A  +/-   PIM   SOG    FW   HIT
my-scores    5.2   9.1 -1.2  17.6  62.2  59.0  34.7
opponent     6.8  12.0  2.1  11.6  58.9  81.1  27.7
difference  -1.6  -3.0 -3.3   6.0   3.4 -22.1   7.0
mean-opp     6.0  10.6  0.5  14.6  60.5  70.1  31.2
mean-league  7.6  13.4  2.0  13.7  70.6  64.8  29.3
std dev      1.1   2.5  1.4   2.1   7.9  15.9   5.1
num_stds    -1.4  -1.2 -2.3   2.9   0.4  -1.4   1.4
score       -0.3  -0.3 -6.7   0.4   0.1  -0.3   0.2
win_loss    -1.0  -1.0 -1.0   1.0   1.0  -1.0   1.0
Score: -0.29


In [63]:

# lets do second day of week, lots of guys can play
game_day = manager.week[1].to_pydatetime().date()
teams_playing_today = Scraper()._teams_playing_one_day(game_day)
my_roster = manager.my_team.roster().copy()
game_day_players = my_roster[my_roster.team_id.isin(teams_playing_today)]

game_day_players.head()
game_day_players = game_day_players[["eligible_positions", "name", "fpts"]]
game_day_players.head(20)

,eligible_positions,name,fpts
player_id,,,
3358,[D],Brent Burns,0.394861
4684,[LW],Evander Kane,1.135035
5697,"[C, LW]",Tomas Hertl,0.573239
5986,[D],Darnell Nurse,0.687115
7537,[RW],Kailer Yamamoto,-0.181922
4263,"[C, RW]",Mikael Backlund,0.273833
4687,[C],Nazem Kadri,0.658630
7155,[D],Samuel Girard,-0.425131
6865,"[LW, RW]",Conor Garland,-0.177034


In [67]:
opponent_scores_for_day = manager.opponent.scores()
opponent_scores_for_day.head()

# opponent_scores_for_day[opponent_scores_for_day.play_date == game_day].head()

datetime.date(2021, 1, 26)

In [51]:
# availables.eligible_positions.map(set(['LW']).issubset)

def score(goals, assists):
    opponent_goals = 6.8
    oppoonent_assists = 12

    
prob = pulp.LpProblem('NHL', LpMaximize)

player_vars = {player_id:pulp.LpVariable(str(player_id), cat='Binary') for player_id in game_day_players.index.values}

#adds objective function
prob += (lpSum(game_day_players.loc[p_id, 'fpts']*player_vars[p_id] for p_id in game_day_players.index.values))

prob += (lpSum(self.positions.loc[p_id,'C']*player_vars[p_id] for p_id in game_day_players.index.values) == 2)


In [57]:


# d = {i:[] for i in pd.Series(game_day_players['eligible_positions']).unique()}


# pos_num_available = {
#     "C": 2,
#     "LW": 2,
#     "RW": 2,
#     "D": 4,
# }

# for pos in pos_num_available.keys():
#     available_pos = availables[availables.eligible_positions == pos]
#     salary = list(available_pos[["displayName","salary"]].set_index("displayName").to_dict().values())[0]
#     point = list(available_pos[["displayName","points"]].set_index("displayName").to_dict().values())[0]
#     salaries[pos] = salary
#     points[pos] = point


player_id
3358         [D]
4684        [LW]
5697     [C, LW]
5986         [D]
7537        [RW]
4263     [C, RW]
4687         [C]
7155         [D]
6865    [LW, RW]
4984         [C]
Name: eligible_positions, dtype: object